## **Линейная регрессия**

### **Градиентный Спуск**

Существует несколько вариантов для выбора **гипераметров** линейной регрессии посмотрим
- Через прямые матричные операции
- **Через градиентны спуск** (посмотрим здесь)


### **Предсказание цены жилья**

- Загружаем датасет про цены домов в Бостоне; нужно предсказать медиану цены жилья в 506 районах
- Это задача регрессии (предсказываем непрерывные числа)
- Для оценки модели будем использовать RMSE, MSE
- Проверим обобщающую способность модели с помощью train_test_split

In [24]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv')
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [32]:
# работаем с тензорами без req_grad
import torch

y = df['medv']
X = df.drop(['medv'],axis=1)

# Стандартизируем наши данные;
# алгоритм работает стабильнее когда данные стандартизированные
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Добавляем вектор из единиц к нашим данным
X = np.hstack([np.ones(X.shape[0])[:,None], X])

X = torch.tensor(X)
y = torch.tensor(y.values)

### **Использованием методов оптимизации**

Для реализации линейной регрессии с помощью методов оптимизации будем использовать функцию ошибки **среднего квадратичного**, которая является выпуклой функцией в n-мерном пространстве $\mathbb{R}^n$ и в общем виде выглядит следующим образом:
$$MSE = \frac{1}{n} * \sum_{i=1}^{n}{(y_i - a(x_i))^2}.$$

- $x_i$ — вектор-признак $i$-го объекта обучающей выборки
- $y_i$ — истинное значение для $i$-го объекта,
- $a(x)$ — алгоритм, предсказывающий для данного объекта $x$ целевое значение
- $n$ — кол-во объектов в выборке

В случае линейной регрессии $MSE$ представляется как:
$$MSE(X, y, \theta) = \frac{1}{2n} * \sum_{i=1}^{n}{(y_i - \theta^Tx_i)^2} = \frac{1}{2n} \lVert{y - X\theta}\rVert_{2}^{2}=\frac{1}{2n} (y - X\theta)^T(y - X\theta),$$

- $\theta$ — параметр модели линейной регрессии
- $X$ — матрица объекты-признаки
- $y$ - вектор истинных значений, соответствующих $X$

Возьмем первый вариант представления функции ошибки и **посчитаем ее градиент по параметру $\theta$**, предварительно переименовав $MSE$ в $L$:
$$L=\frac{1}{2n} * \sum_{i=1}^{n}{(y_i - \theta^Tx_i)^2}$$
$$\nabla L = \frac{1}{n}\sum_{i=1}^{n}{(\theta^Tx_i - y_i) \cdot x_i} = \frac{1}{n}X^T(X\theta - y)$$

In [33]:
from torch import matmul

# 1) вычисления градиента функции MSE
# Это самое важное dL/dtheta для MSE

def calc_mse_gradient(X,y,theta):
    n = X.size()[0]
    val = matmul(X,theta)-y
    grad = (1/n) * matmul(X.transpose(1,0),val)
    return grad

# calc_mse_gradient(X,y,theta)

# 2) Шаг градиентного спуска
def gradient_step(theta, theta_grad, alpha):
    return theta - alpha * theta_grad

# 3) Процедура оптимизации параметров модели

def optimize(X, y, grad_func, start_theta, alpha, n_iters):

    # начальное theta (theta0)
    theta = start_theta.clone()

    for i in range(n_iters):
        theta_grad = grad_func(X, y, theta) # dL/dtheta
        theta = gradient_step(theta, theta_grad, alpha) # Новый градиент

    return theta


In [35]:
# Оптимизировать параметр линейной регрессии theta на всех данных
m = X.shape[1]                             # размерность задачи
theta = optimize(X,
                 y,
                 calc_mse_gradient,        # функция для вычисления градиента
                 torch.tensor(np.ones(m)), # Начальное theta
                 0.01,                    #
                 500)                      # количество итерации
print(theta)

tensor([22.3913, -0.6193,  0.5859, -0.2119,  0.7838, -0.7920,  3.3627, -0.0629,
        -1.7392,  0.8547, -0.6576, -1.7481,  0.9747, -3.3063],
       dtype=torch.float64)


In [38]:
# Сделать предсказания при полученных параметрах
y_pred = matmul(X,theta)
y_pred[:10]

tensor([30.2735, 25.0164, 31.0352, 29.4691, 29.0785, 25.6656, 23.0928, 20.3691,
        12.5569, 19.8964], dtype=torch.float64)

Посчитать значение ошибку RMSE для тренировочных данных

In [40]:
# Функция для вычисления RMSE, в торче нет RMSE
# и не советуется просто брать nn.sqrt от MSELoss

import torch
from torch import nn

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [41]:
from torch.nn import MSELoss
import torch

def print_regression_metrics(y_true,y_pred):
    criterion_mse = MSELoss()
    criterion_rmse = RMSELoss()
    loss_mse = criterion_mse(y_true,y_pred)
    loss_rmse = criterion_rmse(y_true,y_pred)
    print(loss_mse.item(),loss_rmse.item())

print_regression_metrics(y,y_pred)

23.065293347217054 4.802634105073699


#### **Проверим обобщающую способность модели**

- Мы обучили модель на некой выборке, и на них же посчитали метрику качества
- Мы не проверили обобщающую способность модели
- Разбиваем выборку на train/valid, вычисляем theta
- Делаем предсказание и вычисляем ошибки

In [42]:
from sklearn.model_selection import train_test_split

# Разделяем выборку та тестовую и валидационную часть
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.5)

# вычисляем theta на train
# theta = linreg_linear(X_train, y_train)

# Оптимизировать параметр линейной регрессии theta на всех данных
m = X.shape[1]                             # размерность задачи
theta = optimize(X_train,
                 y_train,
                 calc_mse_gradient,        # функция для вычисления градиента
                 torch.tensor(np.ones(m)), # Начальное theta
                 0.01,                    #
                 500)                      # количество итерации

# Предсказание у нас через matmul
y_pred_tr = matmul(X_train,theta)
y_pred_v = matmul(X_valid,theta)


In [43]:
# Оцениваем качество модели
print_regression_metrics(y_train,y_pred_tr)
print_regression_metrics(y_valid,y_pred_v)

21.916318730774698 4.68148691451495
26.269302806950876 5.125358895428775


В sklearn есть реализация подобного подхода (для удобства)

In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_valid)
mean_squared_error(y_pred,y_valid)


24.201177136887257

### **Заключение**

В этом ноутбуке мы рассмотрели как можно с помощью градиентного спуска оптимизировать параметры модели линейной регрессии, сделали мы это на примере задачи регрессии, в которой мы предсказали цены на жильё